# Average Lunar Flux Brightness Simulation

In [1]:
# import various packages
import numpy as np
# import cupy as cp

## Inititalize Facet Regions

Let's split up the moon into facets along lines of latitude and longitude 

Only necessary to structure the latitude from 0 to 90 degrees because of the symmetry of the northern/southern hemispheres, will need to multiply the average flux density by 2 at the end of the simulation

In [2]:
# Define ranges for phase, longitude, latitude, and declination
# phase_array = np.arange(0, 360, dtype = np.int16) # angular positions of the sun overhead w.r.t latitude = 0, longitude = 0 position
longitude_array= np.arange(-180, 181, dtype = np.int16)  # setting longitude range in degrees
latitude_array = np.arange(0, 91, dtype = np.int16)  # setting latitude range in degrees, (0,90 exclusive) I'm considering the centre point of each facet
declination = 0  # The angular position of the sun at solar noon in degrees

## Measure Facets Areas

In [50]:
def lunar_facets_areas(latitude_array, longitude_array, moon_radius):
    """
    Generation of facet areas over the surface of the Moon using latitude and longitude arrays.
    
    - Parameters:
        - latitude_array: Array of latitudes in degrees.
        - longitude_array: Array of longitudes in degrees.
        - moon_radius: Radius of the Moon in meters.
    
    - Returns: facet_areas: 2D array of facet areas (m²) corresponding to each latitude and longitude grid cell.
    """
    
    # Convert latitude and longitude arrays from degrees to radians
    latitudes = np.radians(latitude_array)
    longitudes = np.radians(longitude_array)
 
    # Compute the areas of the facets
    facet_areas = (moon_radius**2) * np.abs(-1* np.sin(latitudes[np.newaxis, 1:]) + np.sin(latitudes[np.newaxis, :-1])) * (longitudes[1:, np.newaxis] - longitudes[:-1, np.newaxis])
    
    return facet_areas

### Test of generate_moon_facets_areas
If the radius is 1 the sum of all facet areas should be 2pi for a hemisphere

In [51]:
moon_radius_test = 1  # Radius of the Moon in meters
area_of_facets_test = lunar_facets_areas(latitude_array, longitude_array, moon_radius_test)
print('shape = ', area_of_facets_test.shape, 'area of surface = ' , np.sum(area_of_facets_test), ', Difference: 2pi - area_of_facets = ', 2 * np.pi - np.sum(area_of_facets_test))

shape =  (360, 90) area of surface =  6.283185 , Difference: 2pi - area_of_facets =  4.7683716e-07


### Implement the funciton with the radius of the Moon in meters

In [52]:
moon_radius = 1.737e6  # Radius of the Moon in meters
area_of_facets = lunar_facets_areas(latitude_array, longitude_array, moon_radius)

In [53]:
np.abs(2*np.pi*moon_radius**2 - np.sum(area_of_facets)), np.sqrt(np.abs(2*np.pi*moon_radius**2 - np.sum(area_of_facets)))

(np.float32(0.0), np.float32(0.0))

The area is off by a bit, how much? a grid approximately 257 m * 257 m in area, I think thats acceptable.

## Solar Zenith Angle for Phase cycle

In [54]:
# def calculate_zenith_angle(phase, latitude, longitude, declination):
#     """
#     Vectorized calculation of the zenith angle on each facet of lat, lon.
#     - Parameters: 
#         - Latitude: a numpy array of latitudes
#         - Longutide: a numpy array of longitudes
#         - Phase: a numpy array of phases
#     - Returns: Outputs a 3D array of zenith angles with shape (360, 360, 91)
#     """
#     # Convert angles from degrees to radians for numpy trigonometric functions
#     phase_rad = np.radians(phase)[:, np.newaxis, np.newaxis]  # shape (360, 1, 1)
#     longitude_rad = np.radians(longitude)[np.newaxis, :, np.newaxis]  # shape (1, 360, 1)
#     latitude_rad = np.radians(latitude)[np.newaxis, np.newaxis, :]  # shape (1, 1, 91)
#     declination_rad = np.radians(declination)

#     # Calculate the zenith angle for all combinations of phase, lon, and lat
#     zenith_angle_array = np.arccos(
#         np.sin(declination_rad) * np.sin(latitude_rad)
#         + np.cos(declination_rad) * np.cos(latitude_rad)
#         * np.cos(longitude_rad - phase_rad)
#     )

#     return zenith_angle_array

In [55]:
def calculate_zenith_angle(latitude, longitude, declination):
    """
    Vectorized calculation of the zenith angle on each facet of lat, lon.
    - Parameters: 
        - Latitude: a numpy array of latitudes
        - Longutide: a numpy array of longitudes
        - Phase: a numpy array of phases
    - Returns: Outputs a 3D array of zenith angles with shape (360, 360, 91)
    """
    # Convert angles from degrees to radians for numpy trigonometric functions
    # phase_rad = np.radians(phase)[:, np.newaxis, np.newaxis]  # shape (360, 1, 1)
    longitude_rad = np.radians(longitude)[:, np.newaxis]  # shape (1, 360, 1)
    latitude_rad = np.radians(latitude)[np.newaxis, :]  # shape (1, 1, 91)
    declination_rad = np.radians(declination)

    # Calculate the zenith angle for all combinations of phase, lon, and lat
    zenith_angle_array = np.arccos(
        np.sin(declination_rad) * np.sin(latitude_rad)
        + np.cos(declination_rad) * np.cos(latitude_rad)
        * np.cos(longitude_rad)
    )

    return zenith_angle_array

In [57]:
# Call the function
zenith_angle_array = calculate_zenith_angle(latitude_array,
                                            longitude_array,
                                            declination)

# check the shape
print(zenith_angle_array.shape)

(361, 91)


#### zenith_angle sanity checks

In [58]:
# np.degrees(zenith_angle_array[0,180,:]) 
# sanity check for what the latitude angles would look like for a sub solar position at theta = 0, longitude, = 0, and phase = 0

In [59]:
# np.degrees(zenith_angle_array[0,225,:]) 
# sanity check for what the latitude angles would look like for longitude = 45, and phase = 0

In [60]:
# np.degrees(zenith_angle_array[45,225,:]) 
# sanity check for what the latitude angles would look like for longitude = 45, and phase = 45

In [61]:
# np.degrees(zenith_angle_array[0,270,:]) 
# sanity check for what the latitude angles would look like for longitude = 90, and phase = 0

In [62]:
# np.degrees(zenith_angle_array[0,0,:]) 
# sanity check for what the latitude angles would look like for longitude = 0, and phase = 0

In [63]:
# vector_zenith_angle_array = vector_calculate_zenith_angle(phase_array,
#                                                           latitude_array,
#                                                           longitude_array,
#                                                           declination)
# zenith_angle_array.shape # checking shape

In [64]:
# np.sum(vector_zenith_angle_array) == np.sum(zenith_angle_array) 
# because they are the essentially the same function, they should result in the same sum of angles

In [65]:
# zenith_angle_array[1,:,:].shape

## Calculate Heat Input

In [66]:
# def flux_input_func(W_0, zenith_angle_array, albedo, phase_step):
#     """
#     - Parameters:
#         - W_0: energy flux W / m**2
#         - zenith_angle_array: array of incident vector angle w.r.t sun
#         - albedo: Lunar Albedo
#         - phase_step: phase ranges from [0,360) degrees
    
#     -Return: the flux input from the Sun. The time is indexed in degrees of phase, which allows isolating the values
#     of latitude and longitude at that specific phase.
#     """
#     # Calculate the heat input based on the zenith angle
#     W_t = W_0 * np.clip(np.cos(zenith_angle_array[phase_step, :, :]), 0, None) * (1 - albedo)
#     # the clip returns a 0 for any values less than 0 resulting from an angle greater than 90 degrees
#     # this way no negative heat is introduced.
    
#     return W_t
               


In [67]:
def flux_input_func(W_0, zenith_angle_array, albedo):
    """
    - Parameters:
        - W_0: energy flux W / m**2
        - zenith_angle_array: array of incident vector angle w.r.t sun
        - albedo: Lunar Albedo
    
    -Return: the flux input from the Sun. The time is indexed in degrees of phase, which allows isolating the values
    of latitude and longitude at that specific phase.
    """
    # Calculate the heat input based on the zenith angle
    W_t = W_0 * np.clip(np.cos(zenith_angle_array), 0, None) * (1 - albedo)
    # the clip returns a 0 for any values less than 0 resulting from an angle greater than 90 degrees
    # this way no negative heat is introduced.
    
    return W_t
               


#### Solar Irradiance Test

In [68]:
W_0 = 1365.0 # Solar irradiance in W.m-2, the energy flux arriving from the Sun
albedo = 0.15 # the lunar albedo
phase_step = 0 # One degree in phase corresponds to 6700 seconds, phase ranges from [0,360) degrees
                # corresponds to [0, 2,405,300 seconds), [0, 668 hours), [0,27.8 days) phase_step inputs must be integers

flux_input_array = flux_input_func(W_0, 
                                   zenith_angle_array, 
                                   albedo)

In [121]:
# shape check
flux_input_array.shape
# type(flux_input_array[0][0])

(361, 91)

In [70]:
# verify that its matching the expected value
print('function output', flux_input_array[180,:][0], 'manual calculation', (1-albedo) * 1365)
# sanity check for when longitude = 0, latitude = 0
# if time = 0 then subsolar position is theta = 0, longitude = 0

function output 1160.25 manual calculation 1160.25


### Test of Lunar Flux

Combining the functions from flux input and facet areas

In [71]:
area_of_facets.shape, flux_input_array.shape

((360, 90), (361, 91))

If we are instead interested in the centres of the facets, the flux input needs its ends trimed off the last values in the longitude and latitude values

In [72]:
lunar_total_flux = area_of_facets * flux_input_array[:-1,:-1]

In [73]:
# # test of the 0 longitude positions
# lunar_total_flux[180]

In [74]:
np.sum(lunar_total_flux)

np.float64(5528970293817895.0)

## Define Thermal Energy Tranfer Functions

In [114]:
def rate_of_change_surface_thermal_energy_func(flux_input_array, area_of_facets, T_0, T_1, sigma, BasCond, tau):
    """
    A function to calculate the rate of change of thermal energy at the surface 
    - Parameter: 
        - heat_input_array: a multi-dimensional array
        - T_0: temperatue at surface position
        - T_1: temperature at depth below surface
        - sigma: Stefan's constant 5.67e-8 (W/m²K⁴)
        - BasCond: 1500, Basalt conductivity in W.m-3.K-1
        - tau: 0.01, is the thickness of the slice [m]
        
    -Return: A scalar value of the rate of change of thermal energy of the surface layer
    """
    # previous version
    # dQ0_dt = heat_input_array - sigma * T_0**4 + (T_1 - T_0) / R 

    # Ken's Version
    # LTK[j] =  LTK[j] + (A*(I - sigma*LTK[j]**4) + (LTK[j+1] - LTK[j])*beta)*delta_t/gamma        
    print("flux_input_array shape:", flux_input_array.shape)
    print("area_of_facets shape:", area_of_facets.shape)
    print("T_0 shape:", T_0.shape)
    print("T_1 shape:", T_1.shape)

    # New Version
    dQ0_dt = area_of_facets * (flux_input_array - sigma * T_0**4) + ((T_1 - T_0) * BasCond * area_of_facets/tau)

    return dQ0_dt

In [76]:
# beta = 0.15         #BasCond * A/delta_z         #conductivity parameter
# gamma = 10050 #BasCap * m                 #thermal capacity parameter
# BasCond = 1.50e3                 #Basalt conductivity in W.m-3.K-1
# BasCap  = 670.0                  #Basalt thermal capacity in J Kg-1.K-1 
# m = A * delta_z * rho            #mass of element
# rho     = 1500.0                 #Lunar regolith density in Kg.m-3
# delta_z = 0.01                   #thickness of element (metres)


In [77]:
def rate_of_change_intermediate_thermal_energy_func(T_np1, T_n, T_nm1, BasCond, area_of_facets, tau):
    """
    A function to calculate the rate of change of thermal energy in the intermediate layers
    - Parameter: 
        - T_nm1: Previous Layer (T_n-1)
        - T_np1: Next Layer (T_n+1)
        - T_n: Layer being evaluated
        - beta:
        
    -Return: A scalar value of the rate of change of thermal energy of the surface layer applied to each element for which there 
    """
    # previous version
    # dQn_dt = ((T_np1 - T_n) / R) + ((T_nm1 - T_n) / R)

    # Ken's Version from moon_temp_model
    # LTK[j] = LTK[j] + ((LTK[j+1]-LTK[j])*beta + (LTK[j-1] - LTK[j])*beta)*delta_t/gamma 

    dQn_dt = ((T_np1 - T_n) + (T_nm1 - T_n)) * (BasCond * area_of_facets/tau)

    
    return dQn_dt

In [78]:
def rate_of_change_final_thermal_energy_func(T_const, T_f, T_nm1, BasCond, area_of_facets, tau):
    """
    The rate of change of thermal energy at the final depth 
    - Parameter: 
        - T_nm1: Previous Layer (T_n-1)
        - T_const: The temperature below the layers being evaluated, a constant value
        - T_f: The final layer being evaluated that between the previous layer and the T_constant layer
        - R: Thermal Resistance
    -Return: A scalar value of the rate of change of thermal energy of the surface layer applied to each element for which there 

    """
    dQf_dt = ((T_const - T_f) + (T_nm1 - T_f)) * (BasCond * area_of_facets/tau)
    
    return dQf_dt

In [79]:
# old version
# def thermal_capacitance(beta, area, tau):
#     """
#     Calculation of the thermal capacitance
#     - Parameters:
#         - beta: the thermal capacity per unit volume
#         - area: is the area of the slice of material
#         - tau: is the thickness of the slice
#     - Return: Q, thermal capacitance [J/K]
#     """
#     Q = beta * area * tau
    
#     return Q

In [80]:
# New Version
def thermal_capacitance(BasCap, area, tau, density):
    """
    Calculation of the thermal capacitance
    - Parameters:
        - BasCap: Basalt thermal capacity in J Kg-1.K-1 
        - area: is the area of the slice of material, passed as an array of area's
        - tau: 0.01, is the thickness of the slice [m]
        - density: 1500.0, Lunar regolith density in Kg.m-3
    - Return: gamma, thermal capacitance [J/K]
    """
    gamma = BasCap * area * tau * density
    
    return gamma

In [81]:
# not actually used in the function, simplified to output temperature

# def thermal_charge(Q_i, dQi_dt, dt):
#     """
#     The thermal charge, Q in one of the layers is updated where,
#     Q_i is the presently evaluated thermal charge,
#     dQi_dt is the rate of change of thermal energy,
#     dt is the time interval
#     """
#     Q_i += dQi_dt * dt

## Develop the Heat Transfer Model

In [82]:
# # Heat transfer model function
# def heat_transfer_model_func(heat_transfer_array, W_0, albedo, T_const, BasCap, area_of_facets, tau, rho, delta_t, model_run_time):
#     """
#     Heat transfer model to update temperature in each layer over time.

#     Parameters:
#     - heat_transfer_array: 3D NumPy array representing solar flux with dependance on incident angle
#     - W_0: Energy flux arriving from the Sun
#     - albedo: Albedo (reflectivity) of the surface
#     - T_const: Constant temperature for the final boundary condition
#     - R: Thermal resistance between layers
#     - beta
#     - area
#     - tau
#     - delta_t: Time step for updating temperatures
#     - model_run_time: Total simulation time in seconds

#     Returns:
#     - Updated heat_transfer_array with new temperatures for each layer.
#     """
#     phase_step = 0
    
#     for second in range(0, model_run_time, delta_t): # range(start,stop,step)
        
#         # Step 1: Initialize Thermal Capacitance
#         gamma = thermal_capacitance(BasCap, area_of_facets, tau, rho)

        
#         # Step 2: Update the surface layer (T_0)
#         T_0 = heat_transfer_array[phase_step,:,:,0]
#         T_1 = heat_transfer_array[phase_step,:,:,1]
#         W_t = flux_input_func(W_0, zenith_angle_array, albedo, phase_step)
#         dQ0_dt = rate_of_change_surface_thermal_energy_func(W_t[:-1,:-1], area_of_facets, T_0, T_1, sigma, BasCond, tau)
#         T_0 += dQ0_dt * delta_t / gamma


#         # Step 3: Update intermediate layers
#         T_n = heat_transfer_array[phase_step,:,:,1:-1]  # All intermediate layers
#         T_np1 = heat_transfer_array[phase_step,:,:,2:]  # Next layer in each direction
#         T_nm1 = heat_transfer_array[phase_step,:,:,:-2]  # Previous layer in each direction
#         dQn_dt = rate_of_change_intermediate_thermal_energy_func(T_np1, T_n, T_nm1, BasCond, area_of_facets[:,:,np.newaxis], tau)
#         T_n += dQn_dt * delta_t / gamma[:,:,np.newaxis]

#         # Step 4: Update the final layer (T_f)
#         T_f = heat_transfer_array[phase_step,:,:,-1]
#         T_fm1 = heat_transfer_array[phase_step,:,:,-2]
#         T_nm1 = T_nm1[:,:,-1]
#         dQf_dt = rate_of_change_final_thermal_energy_func(T_const, T_f, T_nm1, BasCond, area_of_facets, tau)
#         T_f += dQf_dt * delta_t / gamma

#         # Step 5: Update the heat transfer array with new temperatures
#         heat_transfer_array[phase_step,:,:,0] = T_0
#         heat_transfer_array[phase_step,:,:,1:-1] = T_n
#         heat_transfer_array[phase_step,:,:,-1] = T_f

#         # uncomment for diagnosing
#         # print(f"At second {second}: T_0 = {T_0}, T_1 = {T_1}, W_t = {W_t}, dQ0_dt = {dQ0_dt}")
#             # Independent phase_step increment based on the condition
#         # if second % 6700 == 0:
#         #     phase_step += 1
#     return heat_transfer_array


In [136]:
# Heat transfer model function
def heat_transfer_model_func(heat_transfer_array, W_0, albedo, T_const, BasCap, area_of_facets, tau, rho, delta_t, model_run_time):
    """
    Heat transfer model to update temperature in each layer over time.

    Parameters:
    - heat_transfer_array: 3D NumPy array representing solar flux with dependance on incident angle
    - W_0: Energy flux arriving from the Sun
    - albedo: Albedo (reflectivity) of the surface
    - T_const: Constant temperature for the final boundary condition
    - R: Thermal resistance between layers
    - beta
    - area
    - tau
    - delta_t: Time step for updating temperatures
    - model_run_time: Total simulation time in seconds

    Returns:
    - Updated heat_transfer_array with new temperatures for each layer.
    """
    
    for second in range(0, model_run_time, delta_t): # range(start,stop,step)
        
        # Step 1: Initialize Thermal Capacitance
        gamma = thermal_capacitance(BasCap, area_of_facets, tau, rho).astype(np.float64)

        
        # Step 2: Update the surface layer (T_0)
        T_0 = heat_transfer_array[:,:,0] # (longitude, latitude, depth)
        T_1 = heat_transfer_array[:,:,1]
        W_t = flux_input_func(W_0, zenith_angle_array, albedo) # (longitude, latitude)/(361, 91) flux as a f() of irradiance, sza, albedo
        # remove last values of W_t
        dQ0_dt = rate_of_change_surface_thermal_energy_func(W_t[:-1,:-1], area_of_facets, T_0, T_1, sigma, BasCond, tau) 
        T_0 += dQ0_dt * delta_t / gamma


        # Step 3: Update intermediate layers
        T_n = heat_transfer_array[:,:,1:-1]  # All intermediate layers
        T_np1 = heat_transfer_array[:,:,2:]  # Next layer in each direction
        T_nm1 = heat_transfer_array[:,:,:-2]  # Previous layer in each direction
        dQn_dt = rate_of_change_intermediate_thermal_energy_func(T_np1, T_n, T_nm1, BasCond, area_of_facets[:,:,np.newaxis], tau)
        T_n += dQn_dt * delta_t / gamma[:,:,np.newaxis]

        # Step 4: Update the final layer (T_f)
        T_f = heat_transfer_array[:,:,-1]
        T_fm1 = heat_transfer_array[:,:,-2]
        T_nm1 = T_nm1[:,:,-1]
        dQf_dt = rate_of_change_final_thermal_energy_func(T_const, T_f, T_nm1, BasCond, area_of_facets, tau)
        T_f += dQf_dt * delta_t / gamma

        # Step 5: Update the heat transfer array with new temperatures
        heat_transfer_array[:,:,0] = T_0
        heat_transfer_array[:,:,1:-1] = T_n
        heat_transfer_array[:,:,-1] = T_f

        # uncomment for diagnosing
        # print(f"At second {second}: T_0 = {T_0}, T_1 = {T_1}, W_t = {W_t}, dQ0_dt = {dQ0_dt}")
            # Independent phase_step increment based on the condition
        # if second % 6700 == 0:
        #     phase_step += 1
    return heat_transfer_array


## Create the heat transfer array and run function

In [116]:
# we know the heat_input_array is of the shape (360,91) for which there are 360 phase degree steps.
# The full phase model will be of shape (360,360,91) and now we want to add a depth component of 100 layers
# This will bring the shape to (360,360,91,100) of which its components are (phase, longitude, latitude, depth)
# of which there is one depth for component for each latitude, longitude pair per phase.
# This turns out to be 1.1 billion elements, lets instead start with a per phase_step

In [144]:
T_const = 225# The initial temperature to which all values are set to initially

# creating an array of (longitude, latitude, depth) filled with T_const
# Ensure the heat_transfer_array is float64 for floating-point operations
# must be run each time to reset the heat_transfer_array 
heat_transfer_array = np.full((360, 90, 100), T_const).astype(np.float64)

# setting various varibles
sigma = 5.67e-8  # Stefan-Boltzmann constant (W/m²K⁴)
W_0 = 1365.0   # the energy flux arriving from the Sun
BasCond = 1.50e3                 #Basalt conductivity in W.m-3.K-1
albedo = 0.15 # lunar albedo
BasCap = 670.0 #Basalt thermal capacity in J Kg-1.K-1 
rho = 1500 # Lunar regolith density in Kg.m-3
tau = 0.01 # tau is the thickness of the slice (meters)
delta_t = 1 # time over which each loop calculation is preformed
model_run_time = 3 # simulation run time in seconds

# run the function 
heat_transfer_model_array = heat_transfer_model_func(heat_transfer_array, 
                                                     W_0, 
                                                     albedo, 
                                                     T_const, 
                                                     BasCap, 
                                                     area_of_facets, 
                                                     tau, 
                                                     rho, 
                                                     delta_t, 
                                                     model_run_time)

heat_transfer_model_array.shape

(360, 90, 100)

In [145]:
area_of_facets.shape

(360, 90)

In [146]:
heat_transfer_model_array[0,0]

array([ -315.74085846, -6766.93589281,  -265.04923199,   176.92463679,
         225.        ,   225.        ,   225.        ,   225.        ,
         225.        ,   225.        ,   225.        ,   225.        ,
         225.        ,   225.        ,   225.        ,   225.        ,
         225.        ,   225.        ,   225.        ,   225.        ,
         225.        ,   225.        ,   225.        ,   225.        ,
         225.        ,   225.        ,   225.        ,   225.        ,
         225.        ,   225.        ,   225.        ,   225.        ,
         225.        ,   225.        ,   225.        ,   225.        ,
         225.        ,   225.        ,   225.        ,   225.        ,
         225.        ,   225.        ,   225.        ,   225.        ,
         225.        ,   225.        ,   225.        ,   225.        ,
         225.        ,   225.        ,   225.        ,   225.        ,
         225.        ,   225.        ,   225.        ,   225.        ,
      

In [102]:
type(thermal_capacitance(BasCap, area_of_facets, tau, rho)[0][0])

numpy.float32

In [87]:
print(heat_transfer_array)


(360, 90, 100)


In [ ]:
# heat_transfer_model_array[0,180,0,:] # Sanity check, values where longitude = 0, theta = 0, and the full depth of 100 layers

## The Radio Brightness Temperature

In [ ]:
def radio_brightness_temp_func(alpha, T_const, heat_transfer_model_array):
    """
    Converting the temperature at each layer to a Temperature Brightness [Kelvin]
    
    Parameters:
    - T_const: The heat at the base layer, does not change
    - alpha: 
    """
    # act upon the bottom layer
    Chi_bottom_layer = alpha * T_const + (1 - alpha) * heat_transfer_model_array[:,:,-1]
    
    # act upon the intermediate layers
    Chi_middle_layers = alpha * heat_transfer_model_array

In [ ]:
def radio_brightness_temp_func(alpha, T_const, heat_transfer_model_array):
    """
    Converts the temperature at each layer to brightness temperature [Kelvin].

    Parameters:
    - alpha: Weighting factor for the brightness temperature formula.
    - T_const: Constant temperature at the base layer (T*).
    - heat_transfer_model_array: 3D array representing temperatures across layers, 
      with the last axis representing the depth (from bottom to top).

    Returns:
    - chi_0: The brightness temperature at the top layer radiated into space.
    - chi_layers: The brightness temperature for all layers.
    """
    
    # Step 1: Get the number of layers (depth is the last axis)
    num_layers = heat_transfer_model_array.shape[2]
    
    # Step 2: Initialize the chi array to hold brightness temperatures for each layer
    chi_layers = np.zeros_like(heat_transfer_model_array)
    
    # Step 3: Calculate the brightness temperature for the bottom layer (last layer in the stack)
    chi_layers[:, :, -1] = alpha * T_const + (1 - alpha) * heat_transfer_model_array[:, :, -1]
    
    # Step 4: From the bottom to the top of the stack
    for n in range(num_layers - 2, -1, -1):
        chi_layers[:, :, n] = alpha * chi_layers[:, :, n+1] + (1 - alpha) * heat_transfer_model_array[:, :, n]
    
    # Step 5: Return the brightness temperature at the top layer (chi_0) and all layers
    chi_0 = chi_layers[:, :, 0]
    return chi_0 # , chi_layers


## The Radio Flux Density

If Ωj is the solid angle subtended by the jth facet as viewed from the Earth, the contribution to the radio flux density is:

In [ ]:
# Constants
k_B = 1.38e-23  # Boltzmann constant in J/K
moon_radius = 1.737e6  # Radius of the Moon in meters
distance_to_earth = 3.844e8  # Average distance from the Moon to Earth in meters

def radio_flux_density(chi_0, solid_angles, wavelength):
    """
    Calculate the radio flux density contribution from each facet.

    Parameters:
    - chi_0: 2D array of brightness temperatures at the top layer for each facet (K).
    - solid_angles: 2D array of solid angles subtended by each facet (sr).
    - wavelength: Wavelength of the radio emission (in meters).

    Returns:
    - total_flux_density: The total radio flux density (W/m²/Hz).
    - delta_S: 2D array of individual contributions to the flux density from each facet.
    """
    
    # Calculate the flux density contribution for each facet using the formula
    delta_S = (2 * k_B * solid_angles * chi_0) / wavelength**2
    
    # Sum over all facets to get the total flux density
    total_flux_density = np.sum(delta_S)
    
    return total_flux_density #, delta_S

In [ ]:
def average_radio_brightness_temperature(total_flux_density, wavelength, solid_angle_moon):
    """
    Calculate the average radio brightness temperature of the lunar disc.

    Parameters:
    - S: Total radio flux density (W/m²/Hz).
    - wavelength: Wavelength of the radio emission (in meters).
    - solid_angle_moon: Solid angle subtended by the Moon (in steradians).

    Returns:
    - T_b: The average radio brightness temperature of the lunar disc (K).
    """
    
    # Calculate the average brightness temperature
    T_b = (total_flux_density * wavelength**2) / (2 * k_B * solid_angle_moon)
    
    return T_b

## Solid Angle Calculations

In [ ]:
def generate_moon_facets_areas_vectorized(latitude_array, longitude_array, moon_radius):
    """
    Generation of facet areas over the surface of the Moon using latitude and longitude arrays.
    
    facet_area = R2×∣sin(latitudei+1)−sin(latitudei)∣×dlongitude
    
    https://math.stackexchange.com/questions/4102850/area-of-surface-between-two-lines-of-latitude
    
    Parameters:
    - latitude_array: Array of latitudes in degrees.
    - longitude_array: Array of longitudes in degrees.
    - moon_radius: Radius of the Moon in meters.
    
    Returns:
    - facet_areas: 2D array of facet areas (m²) corresponding to each latitude and longitude grid cell.
    """
    
    # Convert latitude and longitude arrays from degrees to radians
    latitudes = np.radians(latitude_array)
    longitudes = np.radians(longitude_array)
    
    # Calculate differences in latitude and longitude
    dlat = np.diff(latitudes)[:, np.newaxis]  # Latitude differences (shape: (len(latitudes)-1, 1))
    dlon = np.diff(longitudes)[np.newaxis, :]  # Longitude differences (shape: (1, len(longitudes)-1))
    
    # Compute the areas of the facets
    facet_areas = (moon_radius**2) * np.abs(np.sin(latitudes[1:, np.newaxis]) - np.sin(latitudes[:-1, np.newaxis])) * dlon
    
    return facet_areas


In [ ]:
def calculate_solid_angles_for_moon_with_projection(moon_radius, distance_to_earth, latitude_array, longitude_array):
    """
    Vectorized calculation of the solid angles subtended by all facets on the Moon using latitude and longitude arrays,
    accounting for projection effects.
    
    Parameters:
    - moon_radius: Radius of the Moon (m).
    - distance_to_earth: Distance from the Moon to Earth (m).
    - latitude_array: Array of latitudes in degrees.
    - longitude_array: Array of longitudes in degrees.

    Returns:
    - solid_angles: 2D array of solid angles for each facet (steradians).
    """
    
    # Generate facet areas using the vectorized function
    facet_areas = generate_moon_facets_areas_vectorized(latitude_array, longitude_array, moon_radius)
    
    # Convert latitude and longitude arrays from degrees to radians
    latitudes = np.radians(latitude_array)
    longitudes = np.radians(longitude_array)
    
    # # Get the central latitude and longitude (assuming observer is directly above equator)
    # observer_lat = 0.0  # Observer on Earth looking at the Moon's equator
    # observer_lon = 0.0  # Assume the sub-Earth point is at longitude 0
    
    # Broadcast latitudes and longitudes into 2D grids
    lat_grid, lon_grid = np.meshgrid(latitudes, longitudes, indexing='ij')
    
    # Calculate the unit normal vectors for each facet
    x_facet = np.cos(lat_grid) * np.cos(lon_grid)
    y_facet = np.cos(lat_grid) * np.sin(lon_grid)
    z_facet = np.sin(lat_grid)
    
    # Facet normal vectors (shape: [len(latitudes), len(longitudes), 3])
    facet_normals = np.stack([x_facet, y_facet, z_facet], axis=-1)
    
    # Observer direction vector (assumed to be [0, 0, 1] in this case)
    observer_vector = np.array([0, 0, 1])
    
    # Calculate the cosine of the angle between facet normals and observer vector
    cos_theta = np.dot(facet_normals, observer_vector)  # Dot product gives cos(theta)
    
    # Ensure no negative cosines (only facets facing Earth contribute)
    cos_theta = np.clip(cos_theta, 0, 1)  # Set cos(theta) to 0 for facets not facing Earth
    
    # Calculate solid angles, including the projection factor
    solid_angles = (facet_areas * cos_theta[:-1, :-1]) / distance_to_earth**2
    # solid_angles = (facet_areas * cos_theta) / distance_to_earth**2

    
    return solid_angles


# Latitude and longitude arrays 
latitude_array = np.arange(0, 91)  # degrees
longitude_array = np.arange(-180, 180)  # degrees

# Calculate solid angles
solid_angles_with_projection = calculate_solid_angles_for_moon_with_projection(moon_radius, distance_to_earth, latitude_array, longitude_array)

solid_angles_with_projection.shape  # Display the vectorized solid angles

In [ ]:
solid_angles_with_projection

### Kens moon_temp_model

In [ ]:
import numpy as np

# Model Constants
DELTA_PHASE = 0.05                # Phase increment in degrees
LUNATION_COUNT = 16               # Number of lunations
LUNATION_SECONDS = ((29 * 24 + 12) * 60 + 44) * 60

# File setup for output
fname = "moontemp_phaseloop.csv"

# Physical Constants
PI = np.pi
SIGMA = 5.67e-8                # Stefan-Boltzmann Constant (W/m^2K^4)
BAS_COND = 1.50e3              # Basalt conductivity (W.m-3.K-1)
BAS_CAP = 670.0                # Basalt thermal capacity (J Kg-1.K-1)  
RHO = 1500.0                   # Lunar regolith density (Kg.m-3)
ALBEDO = 0.15                  # Lunar albedo (dimensionless)
SOLAR_CONSTANT = 1365.0        # Solar irradiance (W/m^2)
T_MEAN = 225.0                 # Mean lunar soil temperature in Kelvins
RF_ABSORPTION_COEFF = 1        # RF Transmission Coefficient

# Model Parameters
A = 1.0                        # Surface area of element (square meters)
DELTA_Z = 0.01                 # Thickness of element (meters)
MASS_ELEMENT = A * DELTA_Z * RHO  # Mass of element (kg)

# Derived constants
I0 = (1 - ALBEDO) * SOLAR_CONSTANT  # Solar energy absorbed by lunar surface
BETA = BAS_COND * A / DELTA_Z       # Conductivity parameter
GAMMA = BAS_CAP * MASS_ELEMENT      # Thermal capacity parameter
ALPHA = np.exp(-RF_ABSORPTION_COEFF * DELTA_Z)  # RF Transmission Coefficient

def calculate_zenith_angle(phase):
    """Calculate the zenith angle of the sun based on the phase."""
    zenith_angle = abs(phase - 180)
    return zenith_angle

def calculate_solar_irradiance(phase):
    """Calculate solar irradiance at the surface based on phase."""
    zenith_angle = calculate_zenith_angle(phase)
    if zenith_angle > 90:
        return 0
    irradiance = I0 * np.cos(np.radians(zenith_angle))
    return irradiance

def update_temperature_intermediate_layers(LTK, irradiance, delta_t):
    """Update the temperature of intermediate soil layers."""
    for j in range(29):
        if j == 0:
            # Surface layer with solar irradiance and radiation loss
            LTK[j] += (A * (irradiance - SIGMA * LTK[j] ** 4) + (LTK[j + 1] - LTK[j]) * BETA) * delta_t / GAMMA
        else:
            # Intermediate layers with only conduction
            LTK[j] += ((LTK[j + 1] - LTK[j]) * BETA + (LTK[j - 1] - LTK[j]) * BETA) * delta_t / GAMMA
    return LTK

def calculate_radio_brightness_temperature(LTK):
    """Calculate the radio brightness temperature based on the soil temperature profile."""
    Tb = T_MEAN  # Initial brightness temperature
    for k in range(29, -1, -1):
        Tb = ALPHA * Tb + (1 - ALPHA) * LTK[k]
    LTK[30] = Tb
    return LTK

def soilstack(LTK, phase, delta_phase):
    """Simulate the lunar soil temperature and update brightness temperature."""
    delta_t = delta_phase * LUNATION_SECONDS / 360
    irradiance = calculate_solar_irradiance(phase)
    LTK = update_temperature_intermediate_layers(LTK, irradiance, delta_t)
    LTK = calculate_radio_brightness_temperature(LTK)
    return LTK

# Initialize soil temperatures to T_mean (30 soil temperatures, LTK[0] to LTK[29])
LTK = [T_MEAN] * 31

# Run the lunation and phase loop and write results to a CSV file
with open(fname, 'w') as opfile:
    for lunation in range(LUNATION_COUNT):
        phase = 0.0
        while phase <= 360:
            # Run temperature update for each phase step
            print(f"Running soilstack for lunation {lunation}, phase {phase} degrees")
            LTK = soilstack(LTK, phase, DELTA_PHASE)
            phase += DELTA_PHASE

            # Format temperature values for output
            temp_values = [f"{0.1 * int(10 * temp):.1f}" for temp in LTK]
            opstring = f"{lunation}, {phase:.1f}, " + ", ".join(temp_values)

            # Only output if lunation > 14
            if lunation > 14:
                print(opstring)
                opfile.write(opstring + "\n")

print("Simulation complete")
